# Imports

In [198]:
import sqlite3
import pandas as pd
import re # for regex functionality
import matplotlib.pyplot as plt # Import for visuals
from matplotlib.ticker import FuncFormatter
import seaborn as sns
# need to import data dictionary python file that I created - 
import sys
import os

# Get the absolute path to the directory
module_path = os.path.abspath('../data/meta data')
# Add the directory to sys.path
sys.path.append(module_path)
# Import the dictionary from the Python file
from vehicle_data_descriptions import descript_dict

# Set display options to prevent wrapping columns
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.width', None)

### Read CSV file to data frame and load into sqlite database.

In [199]:
vehicle_data = pd.read_csv('../data/vehicles.csv', dtype={74: str, 75: str, 77: str}) # make sure these specific columns contain string values as they were mixed value types.

# Create/Connect to SQLite database
connection = sqlite3.connect('../data/vehicles_data.db')
vehicle_data.to_sql('vehicles_data_sql', connection, if_exists='replace', index=False)
connection.close()

### Test read data from sqlite db

In [200]:
connection = sqlite3.connect('../data/vehicles_data.db')
df = pd.read_sql_query('SELECT * FROM vehicles_data_sql', connection)
connection.close()

# print(df.head(2))
# print(df.columns)

### List which columns to keep

In [201]:
keep_columns = ['barrels08', 'charge240', 'cityE', 'co2', 'co2TailpipeGpm', 'comb08', 'combE','cylinders', 'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08', 'fuelType', 'fuelType1',  'ghgScore', 'id', 
           'make', 'model', 'mpgData', 'range', 'rangeCity', 'rangeHwy', 'trany', 'VClass', 'year', 'youSaveSpend', 'baseModel', 'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'evMotor', 'mfrCode', 
           'c240Dscr', 'charge240b'] # 37 total columns to keep, possible usable data

### List which fuel types to remove

In [202]:
remove_fuel_types = ['Diesel', 'CNG', 'Gasoline or natural gas', 'Gasoline or E85', 'Gasoline or propane' 'Premium or E85', 'Premium Gas or Electricity', 'Regular Gas and Electricity', 
                     'Premium and Electricity', 'Regular Gas or Electricity', 'Hydrogen', 'Gasoline or propane', 'Premium or E85'] # rows where cars use unwanted fuel types


### Clean initial data - by loading only what I need from the sqlite database.

In [203]:
keep_col_str = ', '.join(keep_columns)
remove_fuel_types_str = ', '.join(f"'{fuel}'" for fuel in remove_fuel_types) # need single quotes for each item

sqlite_query = f"""
SELECT {keep_col_str}
FROM vehicles_data_sql
WHERE fuelType NOT IN ({remove_fuel_types_str})
"""

connection = sqlite3.connect('../data/vehicles_data.db')

loaded_vehicle_data = pd.read_sql_query(sqlite_query, connection)

connection.close()

#loaded_vehicle_data

### Write data to txt file - all unique values for viewing and adjustment.

In [204]:
# Write to txt file so I can view the data
#---------------
spacer = ['-'*20, '-'*2]
#titleList = ['Data Discovery', 'Annual fuel consumption', 'Charging time', 'Mileage/Type', 'Carbon Emissions per mile', 'MPG/MPKwh']

# get names and unique values for each column in the column list
with open('../Documents/Column_Descript_Unique_Values.txt', 'w') as file:
    for col in loaded_vehicle_data:
        unique_values = loaded_vehicle_data[col].unique()
        description = descript_dict.get(col, "No description available")
        file.write(f'{col}:  {description}\n')
        file.write(f"{spacer[1]}\n")
        for value in unique_values:
            file.write(f"{value}, ")
        file.write("\n" + "-"*10 + "\n")

#------------

### Further cleaning data

In [205]:
column_drop_list = ['mfrCode', 'evMotor', 'sCharger', 'tCharger', 'trans_dscr', 'rangeHwy', 'rangeCity', 'mpgData', 'eng_dscr', 'engId', 'cityE', 'co2TailpipeGpm', 'fuelType', 'charge240b']

loaded_vehicle_data = loaded_vehicle_data.drop(columns=column_drop_list)

In [215]:
# rename some columns to provide better context.
loaded_vehicle_data.rename(columns={
    'c240Dscr': 'charger_descript_240', 'charge240': 'charge_hours_240', 'co2': 'co2', 
    'fuelType1': 'fuel_type', 'comb08': 'comb_MPG_MPGe', 'fuelCost08': 'est_ann_fuel_cost',
    'combE': 'comb_kWh_100miles', 'feScore':'epa_FuelEcon_score', 'ghgScore': 'greenhouse_Gas_Score',
    'barrels08': 'est_petro_cons', 'youSaveSpend': '5yr_SaveSpend', 'baseModel': 'base_Model',
    'atvType': 'base_fuel_type'
    }, inplace=True)
print(loaded_vehicle_data.columns)
# print(loaded_vehicle_data.shape)
# print(loaded_vehicle_data.base_Model)
# print(loaded_vehicle_data.model)
# print(loaded_vehicle_data.make)

# c240Dscr to charger_descript_240, charge240 to charge_hours_240, fuelType1 to fuel_type

Index(['est_petro_cons', 'charge_hours_240', 'co2', 'comb_MPG_MPGe',
       'comb_kWh_100miles', 'cylinders', 'displ', 'drive',
       'epa_FuelEcon_score', 'est_ann_fuel_cost', 'fuel_type',
       'greenhouse_Gas_Score', 'id', 'make', 'model', 'range', 'trany',
       'VClass', 'year', '5yr_SaveSpend', 'base_Model', 'base_fuel_type',
       'charger_descript_240', 'year_make_model_key'],
      dtype='object')


In [207]:
# print(loaded_vehicle_data[['co2', 'co2TailpipeGpm']])
# print(loaded_vehicle_data['co2'].unique())
# print(loaded_vehicle_data['co2TailpipeGpm'].unique()) - Removed column

# After viewing in Data Wrangler, co2 and co2TailpipeGpm seem to contain the same data but one is rounded and one contains decimals

#loaded_vehicle_data['base_fuel_type'].fillna('Gasoline', inplace=True) # fill na with Gasoline as that is going to be the base fuel if not electric or hybrid

loaded_vehicle_data['base_fuel_type'] = loaded_vehicle_data['base_fuel_type'].fillna('Gasoline') # fill na with Gasoline as that is going to be the base fuel if not electric or hybrid

#df

### load another data set to combine with existing data

In [208]:
vehicle_sales_data = pd.read_csv('../data/car_prices_dataset/car_prices.csv')

vehicle_sales_data['transmission'].unique()

vehicle_sales_data

#loaded_vehicle_data_ev = loaded_vehicle_data[loaded_vehicle_data['base_fuel_type'] == 'EV']




,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558832,2015,Kia,K900,Luxury,Sedan,NaN,knalw4d4xf6019304,in,45.0,18255.0,silver,black,avis corporation,35300.0,33000.0,Thu Jul 09 2015 07:00:00 GMT-0700 (PDT)
558833,2012,Ram,2500,Power Wagon,Crew Cab,automatic,3c6td5et6cg112407,wa,5.0,54393.0,white,black,i -5 uhlmann rv,30200.0,30800.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558834,2012,BMW,X5,xDrive35d,SUV,automatic,5uxzw0c58cl668465,ca,48.0,50561.0,black,black,financial services remarketing (lease),29800.0,34000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558835,2015,Nissan,Altima,2.5 S,sedan,automatic,1n4al3ap0fc216050,ga,38.0,16658.0,white,black,enterprise vehicle exchange / tra / rental / tulsa,15100.0,11100.0,Thu Jul 09 2015 06:45:00 GMT-0700 (PDT)


### Create a column that will allow the two data sets to combine
- Creating a key that would not have duplicates and would be identicle between the two tables is unlikley to be possible.
- The solution appears to be to do a 1 to many combination that would allow for further aggregation on the key when necessary for visualization.
- I will be able to clean the sales data and aggregate the data to have only one copy of a key to use, I will need to keep many keys on the loaded_vehicle_data.

In [209]:
# Remove spaces and combine the columns into lowercase
loaded_vehicle_data['year_make_model_key'] = (
    loaded_vehicle_data['year'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True) +
    loaded_vehicle_data['make'].str.replace(r'[^a-zA-Z0-9]', '', regex=True) + 
    loaded_vehicle_data['base_Model'].str.replace(r'[^a-zA-Z0-9]', '', regex=True)
).str.lower()

print(loaded_vehicle_data.columns)
print('\n')

key_counts = loaded_vehicle_data['year_make_model_key'].value_counts()
print(key_counts[key_counts > 1])  # Display keys that appear more than once



Index(['est_petro_cons', 'charge_hours_240', 'co2', 'comb_MPG_MPGe',
       'comb_kWh_100miles', 'cylinders', 'displ', 'drive',
       'epa_FuelEcon_score', 'est_ann_fuel_cost', 'fuel_type',
       'greenhouse_Gas_Score', 'id', 'make', 'model', 'range', 'trany',
       'VClass', 'year', '5yr_SaveSpend', 'base_Model', 'base_fuel_type',
       'charger_descript_240', 'year_make_model_key'],
      dtype='object')


year_make_model_key
1988gmcpickup                  47
2019porsche911                 43
2018porsche911                 41
1985gmcpickup                  39
2024porsche911                 38
                               ..
2013hondacrz                    2
2014roushperformancemustang     2
1987volvo780                    2
2013scionxb                     2
1993lexuses                     2
Name: count, Length: 9142, dtype: int64


### do the same with the sales data

In [214]:
# print(vehicle_sales_data.columns)

refined_vehicle_sales_data = vehicle_sales_data[['year', 'make', 'model', 'condition', 'odometer', 'sellingprice']]

# print(vehicle_sales_data.columns)

vehicle_sales_data['year_make_model_key'] = (
    vehicle_sales_data['year'].astype(str).str.replace(r'[^a-zA-Z0-9]', '', regex=True) +
    vehicle_sales_data['make'].str.replace(r'[^a-zA-Z0-9]', '', regex=True) + 
    vehicle_sales_data['model'].str.replace(r'[^a-zA-Z0-9]', '', regex=True)
).str.lower()

# Group by 'year_make_model_key' and calculate the required aggregations
processed_sales_data = vehicle_sales_data.groupby('year_make_model_key', as_index=False).agg(
    avg_selling_price=('sellingprice', 'mean'),  # Average selling price
    avg_odometer_reading=('odometer', 'mean'),  # Average odometer reading
    avg_condition=('condition', 'mean'),  # Average odometer reading
    number_of_sales=('year_make_model_key', 'size')       # Count occurrences, reflects number of sales of that specific year make model
)

# print(processed_sales_data)

key_counts_2 = processed_sales_data['year_make_model_key'].value_counts()
print(key_counts_2[key_counts_2 > 1])  # Display keys that appear more than once


Series([], Name: count, dtype: int64)


In [211]:
filtered_df = loaded_vehicle_data[loaded_vehicle_data['year_make_model_key'].isin(processed_data['year_make_model_key'])]

print(filtered_df.shape)
filtered_df

(14353, 24)


,est_petro_cons,charge_hours_240,co2,comb_MPG_MPGe,comb_kWh_100miles,cylinders,displ,drive,epa_FuelEcon_score,est_ann_fuel_cost,...,model,range,trany,VClass,year,5yr_SaveSpend,base_Model,base_fuel_type,charger_descript_240,year_make_model_key
7,12.396250,0.0,-1,24,0.0,4.0,1.6,Front-Wheel Drive,-1,1950,...,Corolla,0,Automatic 3-spd,Compact Cars,1993,-1500,Corolla,Gasoline,None,1993toyotacorolla
8,11.442692,0.0,-1,26,0.0,4.0,1.6,Front-Wheel Drive,-1,1800,...,Corolla,0,Manual 5-spd,Compact Cars,1993,-750,Corolla,Gasoline,None,1993toyotacorolla
9,11.900400,0.0,-1,25,0.0,4.0,1.8,Front-Wheel Drive,-1,1850,...,Corolla,0,Automatic 4-spd,Compact Cars,1993,-1000,Corolla,Gasoline,None,1993toyotacorolla
10,11.442692,0.0,-1,26,0.0,4.0,1.8,Front-Wheel Drive,-1,1800,...,Corolla,0,Manual 5-spd,Compact Cars,1993,-750,Corolla,Gasoline,None,1993toyotacorolla
23,12.935217,0.0,-1,23,0.0,4.0,2.2,Front-Wheel Drive,-1,2050,...,Century,0,Automatic 3-spd,Midsize Cars,1993,-2000,Century,Gasoline,None,1993buickcentury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44965,11.018889,0.0,-1,27,0.0,4.0,1.8,Front-Wheel Drive,-1,1750,...,Prizm,0,Manual 5-spd,Compact Cars,1993,-500,Prizm,Gasoline,None,1993geoprizm
44966,13.523182,0.0,-1,22,0.0,4.0,2.2,Front-Wheel Drive,-1,2100,...,Accord,0,Automatic 4-spd,Compact Cars,1993,-2250,Accord,Gasoline,None,1993hondaaccord
44968,12.396250,0.0,-1,24,0.0,4.0,2.2,Front-Wheel Drive,-1,1950,...,Accord,0,Manual 5-spd,Compact Cars,1993,-1500,Accord,Gasoline,None,1993hondaaccord
44977,13.523182,0.0,-1,22,0.0,4.0,2.0,Front-Wheel Drive,-1,2100,...,G20,0,Automatic 4-spd,Compact Cars,1993,-2250,G20,Gasoline,None,1993infinitig20


### Join the tables and view the rows that do not match to see what issues may exist with the created key

In [212]:
# Perform a full outer join on the 'make_model_key' column
merged_data = pd.merge(
    loaded_vehicle_data, 
    vehicle_sales_data, 
    on='year_make_model_key', 
    how='outer'
)

merged_data

vehicle_data_ev = merged_data[merged_data['base_fuel_type'] == 'EV']

vehicle_data_ev

ev = merged_data[merged_data['base_fuel_type'] == 'EV']
merged_data
# print(merged_data['base_fuel_type'].unique())

,est_petro_cons,charge_hours_240,co2,comb_MPG_MPGe,comb_kWh_100miles,cylinders,displ,drive,epa_FuelEcon_score,est_ann_fuel_cost,...,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,14.875500,0.0,-1.0,20.0,0.0,6.0,2.5,None,-1.0,2350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.875500,0.0,-1.0,20.0,0.0,6.0,2.5,None,-1.0,2350.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14.167143,0.0,-1.0,21.0,0.0,4.0,2.0,None,-1.0,2200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14.167143,0.0,-1.0,21.0,0.0,4.0,2.0,None,-1.0,2200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16.528333,0.0,-1.0,18.0,0.0,4.0,2.5,4-Wheel or All-Wheel Drive,-1.0,2600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4n2xn11txyd844711,ma,31.0,108900.0,green,gray,boch toyota/scion south,1275.0,1100.0,Thu Jun 18 2015 11:30:00 GMT-0700 (PDT)
2343423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,jtjbt20x770129305,pa,41.0,66393.0,—,beige,r hollenshead auto sales inc,18150.0,18500.0,Fri Jun 19 2015 02:00:00 GMT-0700 (PDT)
2343424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4a4jn2as6be029938,nv,41.0,67820.0,silver,black,imperial rides,9175.0,10500.0,Fri Jun 19 2015 05:00:00 GMT-0700 (PDT)
2343425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1g1yy24u355116011,tx,24.0,114787.0,burgundy,black,hopper motorplex inc,15400.0,14800.0,Wed Jun 24 2015 03:15:00 GMT-0700 (PDT)
